In [ ]:
rm(list=ls())
install.packages("tidyverse"); library(tidyverse)
install.packages("terra"); library(terra)
install.packages("openxlsx");library(openxlsx)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

terra 1.8.10


Attaching package: ‘terra’


The following object is masked from ‘package:tidyr’:

    extract


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# LOAD POPULATION TO HAVE DATA EXTRACTED
linkMetadata<- "https://github.com/Ecosantos/Demogbuff-pops/raw/refs/heads/main/Data/CleanData.RDS"
CleanData<- readRDS(url(linkMetadata, method="libcurl"))


Metadata<-CleanData$Metadata
MetadataClean<-CleanData$MetadataClean

#Reduce (more) data to improve redability
MetadataFinal<-MetadataClean%>%select(-c(lambda,Ecoregion,Binomial))

MetadataFinal<-MetadataFinal%>%
  left_join(.,
            Metadata%>%select(ID,StudyStart, StudyDuration, StudyEnd)%>%distinct(),
            by="ID")


ClimaticData<-Metadata%>%
  dplyr::select(ID,Lat,Lon,SpeciesAccepted,StudyStart,StudyEnd)%>%
  distinct(ID,.keep_all=TRUE)%>%
  filter(!is.na(Lat)&  between(Lat, -90, 90))%>%
  filter(!is.na(Lon) &  between(Lon, -180, 180))

ClimaticData%>%
  arrange(Lon)

#Hlcn.72_716  - Exemplo do brasil +-20ºC

ClimaticData<-ClimaticData%>%
  #filter(SpeciesAccepted=="Clinocottus analis")%>%
  mutate(Lat=case_when(
    SpeciesAccepted=="Clinocottus analis" ~ 	32.82162,
    SpeciesAccepted=="Ursus maritimus" ~    	69.35431,
    SpeciesAccepted=="Nephtys incisa" ~     	41.26105,
    SpeciesAccepted=="Amphimedon compressa" ~ 21.09022,
    TRUE~Lat),
    Lon=case_when(
      SpeciesAccepted=="Clinocottus analis" ~  -117.27783,
      SpeciesAccepted=="Ursus maritimus" ~     -135.44041,
      SpeciesAccepted=="Nephtys incisa" ~      -72.879102,
      SpeciesAccepted=="Amphimedon compressa" ~ 70.792523,
      TRUE~Lon))



sampleraster<-terra::rast("Chelsa sample.tif")


Data_point<-terra::vect(data.frame(ClimaticData[,c(1,3,2)]), geom=c("Lon", "Lat"), crs=crs(sampleraster), keepgeom=TRUE)
Data_point<-terra::project(Data_point, crs(sampleraster))


#=====================================================================================================
# CLIMATIC DATA
#=====================================================================================================

LinksChelsa<-"https://raw.githubusercontent.com/Ecosantos/Demogbuff-pops/main/Data/ChelsacrutsData/ChelsacrutsDataV2.xlsx"

# LOAD LIST OF CLIMATIC DATA TO BE EXTRACTED
Tmin<- openxlsx::read.xlsx(xlsxFile = LinksChelsa, sheet = "Tmin", colNames = TRUE,skipEmptyRows = FALSE)
Tmax<- openxlsx::read.xlsx(xlsxFile = LinksChelsa, sheet = "Tmax", colNames = TRUE,skipEmptyRows = FALSE)
Prec<- openxlsx::read.xlsx(xlsxFile = LinksChelsa, sheet = "Preciptation", colNames = TRUE,skipEmptyRows = FALSE)



Tmin_df<-Tmin%>%
  mutate(param=gsub(Link, pattern=".*/CHELSAcruts_tmin_", replace="\\"))%>%
  mutate(Month=str_split(param, "_", n = 3, simplify = TRUE)[,1])%>%
  mutate(Year=str_split(param, "_", n = 3, simplify = TRUE)[,2])

Tmax_df<-Tmax%>%
  mutate(param=gsub(Link, pattern=".*/CHELSAcruts_tmax_", replace="\\"))%>%
  mutate(Month=str_split(param, "_", n = 3, simplify = TRUE)[,1])%>%
  mutate(Year=str_split(param, "_", n = 3, simplify = TRUE)[,2])

Prec_df<-Prec%>%
  mutate(param=gsub(Link, pattern=".*/CHELSAcruts_prec_", replace="\\"))%>%
  mutate(Month=str_split(param, "_", n = 3, simplify = TRUE)[,1])%>%
  mutate(Year=str_split(param, "_", n = 3, simplify = TRUE)[,2])

#---------------------------------------------------------------------------
# Create data frames to receive extracted information
#---------------------------------------------------------------------------

Prec_df_out<-Tmax_df_out<-Tmin_df_out<-data.frame(matrix(NA, ncol=length(Data_point), nrow=dim(Tmin_df)[1]))
chelsarast<-NULL
destfile<-NULL

ID,Lat,Lon,SpeciesAccepted,StudyStart,StudyEnd
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Alyx.627_74,22.13028,-159.6586,Alyxia stellata,2007,2011
Alyx.627_75,22.20333,-159.6089,Alyxia stellata,2007,2011
Arg..198_130,20.72028,-156.1550,Argyroxiphium sandwicense,1997,2001
Ttrm.21_1392,19.67417,-155.3422,Tetramolopium arenarium,1989,1991
Lgps.394_383,61.26667,-138.5167,Lagopus leucura,2004,2006
Lgps.394_384,61.26667,-138.5167,Lagopus muta,2004,2006
Urs2.183_726,69.50000,-137.0000,Ursus maritimus,2001,2006
Btry.140_91,48.36806,-124.6117,Botrylloides violaceus,2005,2007
Cypr.576_483,42.33333,-123.5167,Cypripedium fasciculatum,1999,2007


In [ ]:
#---------------------------------------------------------------------------
# MINIMUM TEMPERATURE
#---------------------------------------------------------------------------

for( i in 1:dim(Tmin_df)[1]){
  #Determine a name for raster file
  destfile = paste0("CHELSA_tmin_m",
                    Tmin_df$Month[i],"y",Tmin_df$Year[i],".tif")
  #Download raster
  download.file(    Tmin_df$Link[i],destfile,mode="wb")
  #Routine
  chelsarast<-terra::rast(destfile)# Open raster
  Tmin_df_out[i,]<-terra::extract(chelsarast, Data_point, na.rm=TRUE)[,2] #Extract values
  #Delete raster
  file.remove(destfile)
if (i %% 100 == 0) {
    print(i)
  }
  }



[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300


In [ ]:
saveRDS(Tmin_df_out,file="Tmin Chelsa.rds")
